# Part 2: Data Cleaning and Gold Standard Creation

In [29]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional
import re

In [109]:
class Cleansing:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        
        self.duplicate_groups = {
            'account_name': ['Account Name', 'account_name', 'AccountName'],
            'contact_email': ['Contact Email', 'contact_email'],
            'created_date': ['Created Date', 'created_date'],
            'lead_source': ['Lead Source', 'lead_source'],
            'opportunity_amount': ['Opportunity Amount', 'opportunity_amount'],
            'is_active': ['Is Active', 'is_active'],
            'sfdc_id': ['SFDC ID', 'sfdc_id'],
            'annual_revenue': ['Annual Revenue', 'annual_revenue']
        }
    
    
    def is_valid_email(self, email: str) -> bool:
        # Check if email is one of the valid emails
        if pd.isna(email) or email is None:
            return False
        return email in ['help@globex.com', 'contact@acme.com']
    
    def is_placeholder_email(self, email: str) -> bool:
        # Check if email is a placeholder 
        if pd.isna(email) or email is None or email == '':
            return True
        placeholders = ['noemail', 'invalid@', 'user@', 'missing.com', 'placeholder']
        return any(placeholder in str(email).lower() for placeholder in placeholders)
    
    def get_corresponding_account(self, email: str) -> Optional[str]:
        # Get the corresponding account name for a valid email
        if email == 'help@globex.com':
            return 'Globex'
        elif email == 'contact@acme.com':
            return 'Acme Corp'
        return None
    
    def generate_email_for_account(self, account_name: str) -> Optional[str]:
        # Generate corresponding email for Globex or Acme Corp
        if account_name == 'Globex':
            return 'help@globex.com'
        elif account_name == 'Acme Corp':
            return 'contact@acme.com'
        return None
    
    def consolidate_account_and_email(self) -> pd.DataFrame:

        result_df = self.df.copy()
        
        result_df['consolidated_account_name'] = None
        result_df['consolidated_contact_email'] = None
        
        account_columns = ['account_name', 'AccountName', 'Account Name']
        
        for idx, row in result_df.iterrows():

            contact_email_lower_val = row.get('contact_email')
            contact_email_val = row.get('Contact Email')
            
            contact_email_lower_valid = self.is_valid_email(contact_email_lower_val)
            contact_email_valid = self.is_valid_email(contact_email_val)
            
            account_values = {}
            for col in account_columns:
                if col in result_df.columns:
                    account_values[col] = row.get(col)
            
            final_account = None
            final_email = None
            
            for col in account_columns:
                if (col in account_values and 
                    pd.notna(account_values[col]) and 
                    account_values[col] != ''):
                    final_account = account_values[col]
                    break
            
            # If no account found, derive from valid email (all accounts empty case)
            if final_account is None:
                # Check contact_email first (priority)
                if contact_email_lower_valid:
                    final_account = self.get_corresponding_account(contact_email_lower_val)
                    final_email = contact_email_lower_val
                # If contact_email not valid, check Contact Email
                elif contact_email_valid:
                    final_account = self.get_corresponding_account(contact_email_val)
                    final_email = contact_email_val
            
            if final_account is not None and final_email is None:
                email_matched = False
                
                if contact_email_lower_valid:
                    expected_account = self.get_corresponding_account(contact_email_lower_val)
                    if final_account == expected_account:
                        final_email = contact_email_lower_val
                        email_matched = True
                
                if not email_matched and contact_email_valid:
                    expected_account = self.get_corresponding_account(contact_email_val)
                    if final_account == expected_account:
                        final_email = contact_email_val
                        email_matched = True
                
                # If no email match but account is Globex or Acme Corp, generate email
                if not email_matched:
                    generated_email = self.generate_email_for_account(final_account)
                    if generated_email:
                        final_email = generated_email
            
            result_df.at[idx, 'consolidated_account_name'] = final_account
            result_df.at[idx, 'consolidated_contact_email'] = final_email
        
        return result_df

    def consolidate_created_date(self) -> pd.DataFrame:
    
        result_df = self.df.copy()
        
        result_df['consolidated_created_date'] = None
        
        created_date_columns = ['created_date', 'Created Date']
        
        for idx, row in result_df.iterrows():
            final_created_date = None
            
            for col in created_date_columns:
                if col in result_df.columns:
                    raw_date = row.get(col)
                    if (pd.isna(raw_date) or raw_date == '' or 
                        str(raw_date).lower() in ['nat', 'not_a_date', 'none']):
                        continue
                    
                    formatted_date = self.format_date_to_standard(raw_date)
                    if formatted_date:
                        final_created_date = formatted_date
                        break
            
            result_df.at[idx, 'consolidated_created_date'] = final_created_date
        
        return result_df

    def format_date_to_standard(self, date_value) -> Optional[str]:
        if pd.isna(date_value) or date_value is None or date_value == '':
            return None
        
        try:
            parsed_date = pd.to_datetime(date_value, errors='coerce')
            
            if pd.isna(parsed_date):
                return None
            
            return parsed_date.strftime('%Y-%m-%d')
        
        except (ValueError, TypeError):
            return None
    
    def consolidate_lead_source(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_lead_source'] = None
        
        lead_source_columns = ['lead_source', 'Lead Source']
        
        for idx, row in result_df.iterrows():
            final_lead_source = None
            
            for col in lead_source_columns:
                if col in result_df.columns:
                    lead_source_val = row.get(col)
                    if (pd.notna(lead_source_val) and 
                        lead_source_val != '' and 
                        str(lead_source_val).lower() not in ['nat', 'none', 'null']):
                        final_lead_source = lead_source_val
                        break
            
            result_df.at[idx, 'consolidated_lead_source'] = final_lead_source
        
        return result_df
    
    def consolidate_is_active(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_is_active'] = None
        
        is_active_columns = ['is_active', 'Is Active']
        
        for idx, row in result_df.iterrows():
            final_is_active = None
            
            for col in is_active_columns:
                if col in result_df.columns:
                    is_active_val = row.get(col)
                    if (pd.notna(is_active_val) and 
                        is_active_val != '' and 
                        str(is_active_val).lower() not in ['nat', 'none', 'null']):
                        standardized_is_active = self.standardize_is_active(is_active_val)
                        if standardized_is_active is not None:
                            final_is_active = standardized_is_active
                            break
            
            result_df.at[idx, 'consolidated_is_active'] = final_is_active
        
        return result_df
    
    def standardize_is_active(self, is_active_value) -> Optional[bool]:
        if pd.isna(is_active_value) or is_active_value is None or is_active_value == '':
            return None
        
        value_str = str(is_active_value).strip().lower()
        
        if value_str in ['nat', 'none', 'null', '']:
            return None
        
        if value_str in ['true', 't', 'yes', 'y', '1', 'active', 'on']:
            return True
        elif value_str in ['false', 'f', 'no', 'n', '0', 'inactive', 'off']:
            return False
        
        return None
    
    def consolidate_sfdc_id(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_sfdc_id'] = None
        
        sfdc_id_columns = ['SFDC ID', 'sfdc_id']
        
        for idx, row in result_df.iterrows():
            final_sfdc_id = None
            
            for col in sfdc_id_columns:
                if col in result_df.columns:
                    sfdc_id_val = row.get(col)
                    if (pd.notna(sfdc_id_val) and 
                        sfdc_id_val != '' and 
                        str(sfdc_id_val).lower() not in ['nat', 'none', 'null']):
                        standardized_sfdc_id = self.standardize_sfdc_id(sfdc_id_val)
                        if standardized_sfdc_id is not None:
                            final_sfdc_id = standardized_sfdc_id
                            break
            
            result_df.at[idx, 'consolidated_sfdc_id'] = final_sfdc_id
        
        return result_df

    def standardize_sfdc_id(self, sfdc_id_value) -> Optional[str]:
        if pd.isna(sfdc_id_value) or sfdc_id_value is None or sfdc_id_value == '':
            return None
        
        value_str = str(sfdc_id_value).strip()
        
        if value_str.lower() in ['nat', 'none', 'null', '']:
            return None
        
        # Check for placeholder values
        placeholder_patterns = ['abc123', 'xyz-00001', '12345', 'bad_id']
        if value_str.lower() in placeholder_patterns:
            return None
        
        return value_str
    
    def consolidate_monetary(self, field_name: str) -> pd.DataFrame:
        result_df = self.df.copy()
        
        consolidated_column = f'consolidated_{field_name}'
        result_df[consolidated_column] = None
        
        field_columns = self.duplicate_groups.get(field_name, [field_name])
        
        for idx, row in result_df.iterrows():
            final_value = None
            
            for col in field_columns:
                if col in result_df.columns:
                    raw_value = row.get(col)
                    if (pd.notna(raw_value) and 
                        raw_value != '' and 
                        str(raw_value).lower() not in ['nat', 'none', 'null']):
                        standardized_value = self.standardize_monetary(raw_value)
                        if standardized_value is not None:
                            final_value = standardized_value
                            break
            
            result_df.at[idx, consolidated_column] = final_value
        
        return result_df
    
    def standardize_monetary(self, monetary_value) -> Optional[float]:
        if pd.isna(monetary_value) or monetary_value is None or monetary_value == '':
            return None
        
        try:
            value_str = str(monetary_value).strip()
            
            if value_str.lower() in ['nat', 'none', 'null', '', 'not available', 'n/a']:
                return None
            
            # Check if it's already a valid number
            if isinstance(monetary_value, (int, float)) and not pd.isna(monetary_value):
                if monetary_value < 0:
                    return None
                return round(float(monetary_value), 2)
            
            # Word version
            if re.match(r'^[a-zA-Z\s]+$', value_str):
                converted_float = self.convert_text_to_number(value_str)
                if converted_float is None:
                    return None
                if converted_float < 0:
                    return None
                return round(converted_float, 2)
            
            # Remove currency symbols and common formatting for numeric values
            value_str = re.sub(r'[$£€¥₹]', '', value_str)
            value_str = value_str.replace(',', '').replace(' ', '') 
            
            if value_str == '':
                return None
            
            multiplier = 1
            value_str_lower = value_str.lower()
            if value_str_lower.endswith('k'):
                multiplier = 1000
                value_str = value_str[:-1]
            elif value_str_lower.endswith('m'):
                multiplier = 1000000
                value_str = value_str[:-1]
            elif value_str_lower.endswith('b'):
                multiplier = 1000000000
                value_str = value_str[:-1]
            
            try:
                final_float = float(value_str) * multiplier
            except ValueError:
                return None
            
            if final_float < 0:
                return None
            
            return round(final_float, 2)
        
        except (ValueError, TypeError):
            return None

    def convert_text_to_number(self, text_value: str) -> Optional[float]:
        if not text_value:
            return None
            
        text_value = text_value.lower().strip()
        
        number_words = {
            'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5,
            'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10,
            'eleven': 11, 'twelve': 12, 'thirteen': 13, 'fourteen': 14, 'fifteen': 15,
            'sixteen': 16, 'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20,
            'thirty': 30, 'forty': 40, 'fifty': 50, 'sixty': 60, 'seventy': 70,
            'eighty': 80, 'ninety': 90, 'hundred': 100, 'thousand': 1000, 'million': 1000000,
            'billion': 1000000000
        }
        
        try:
            words = text_value.split()
            total = 0
            current = 0
            found_valid_word = False
            
            for word in words:
                word = word.strip()
                if word in number_words:
                    found_valid_word = True
                    value = number_words[word]
                    if value == 100:
                        current = current * 100 if current > 0 else 100
                    elif value >= 1000:
                        total += current * value
                        current = 0
                    else:
                        current += value
            
            total += current
            
            if not found_valid_word or (total == 0 and 'zero' not in text_value):
                return None
            
            return float(total)
        
        except Exception:
            return None

    def consolidate_last_activity(self) -> pd.DataFrame:
        
        result_df = self.df.copy()
        
        result_df['consolidated_last_activity'] = None
        
        last_activity_columns = ['Last Activity']
        
        # Define placeholder values to filter out
        placeholder_values = [
            '42', 42, 'Called Client', 'called client', 'CALLED CLIENT', ''
        ]
        
        for idx, row in result_df.iterrows():
            final_last_activity = None
            
            for col in last_activity_columns:
                if col in result_df.columns:
                    raw_activity = row.get(col)
                    
                    # Skip if NaN, empty, or placeholder value
                    if (pd.isna(raw_activity) or 
                        raw_activity == '' or 
                        raw_activity in placeholder_values or
                        str(raw_activity).strip().lower() in [str(p).lower() for p in placeholder_values]):
                        continue
                    
                    # Try to format as date using existing method
                    formatted_date = self.format_date_to_standard(raw_activity)
                    if formatted_date:
                        final_last_activity = formatted_date
                        break
                    
            
            result_df.at[idx, 'consolidated_last_activity'] = final_last_activity
        
        return result_df
    
    def consolidate_custom_field(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_custom_field'] = None
        
        custom_field_columns = ['Custom Field']
        
        placeholder_values = ['N/A', '{"type": null}', None, '', 'null', 'nat', 'none']
        
        for idx, row in result_df.iterrows():
            final_custom_field = None
            
            for col in custom_field_columns:
                if col in result_df.columns:
                    custom_field_val = row.get(col)
                    
                    if (pd.notna(custom_field_val) and 
                        custom_field_val != '' and 
                        custom_field_val not in placeholder_values and
                        str(custom_field_val).lower() not in [str(p).lower() for p in placeholder_values if p is not None]):
                        final_custom_field = custom_field_val
                        break
            
            result_df.at[idx, 'consolidated_custom_field'] = final_custom_field
        
        return result_df

    def consolidate_region(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_region'] = None
        
        region_columns = ['Region']
        
        north_america_values = ['North America', 'NA', 'N.A.', 'United States', 'US']
        
        for idx, row in result_df.iterrows():
            final_region = None
            
            for col in region_columns:
                if col in result_df.columns:
                    region_val = row.get(col)
                    
                    if (pd.notna(region_val) and 
                        region_val != '' and 
                        str(region_val).strip() in north_america_values):
                        final_region = 'North America'
                        break
            
            result_df.at[idx, 'consolidated_region'] = final_region
        
        return result_df
    
    def consolidate_random_notes(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_random_notes'] = None
        
        random_notes_columns = ['Random Notes']
        
        for idx, row in result_df.iterrows():
            final_notes_flag = None
            
            for col in random_notes_columns:
                if col in result_df.columns:
                    notes_val = row.get(col)
                    
                    # Check if value is "See notes" or "Valid"
                    if (pd.notna(notes_val) and 
                        notes_val != '' and 
                        str(notes_val).strip() in ['See notes', 'Valid']):
                        final_notes_flag = 'Notes_Flag'
                        break
            
            result_df.at[idx, 'consolidated_random_notes'] = final_notes_flag
        
        return result_df
    

    def consolidate_deal_score(self) -> pd.DataFrame:

        result_df = self.df.copy()
        result_df['consolidated_deal_score'] = None
        
        for idx, row in result_df.iterrows():
            deal_score_val = row.get('Deal Score')
            
            if pd.notna(deal_score_val):
                result_df.at[idx, 'consolidated_deal_score'] = deal_score_val / 100.0
            else:
                result_df.at[idx, 'consolidated_deal_score'] = None
        
        return result_df
    
    def consolidate_engagement_level(self) -> pd.DataFrame:
        result_df = self.df.copy()
        result_df['consolidated_engagement_level'] = None
        
        for idx, row in result_df.iterrows():
            engagement_val = row.get('Engagement Level')
            
            if pd.notna(engagement_val):
                result_df.at[idx, 'consolidated_engagement_level'] = round(engagement_val, 4)
            else:
                result_df.at[idx, 'consolidated_engagement_level'] = None
        
        return result_df
    
    
    def consolidate_num_calls(self) -> pd.DataFrame:
        result_df = self.df.copy()
        result_df['consolidated_num_calls'] = None
        
        for idx, row in result_df.iterrows():
            num_calls_val = row.get('Num Calls')
            
            if pd.notna(num_calls_val):
                result_df.at[idx, 'consolidated_num_calls'] = num_calls_val
            else:
                result_df.at[idx, 'consolidated_num_calls'] = None
        
        return result_df
    
    def consolidate_page_duration(self) -> pd.DataFrame:
        result_df = self.df.copy()
        result_df['consolidated_page_duration'] = None
        
        for idx, row in result_df.iterrows():
            time_val = row.get('Time on Page (sec)')
            
            if pd.notna(time_val):
                result_df.at[idx, 'consolidated_page_duration'] = int(time_val)
            else:
                result_df.at[idx, 'consolidated_page_duration'] = None
        
        return result_df
    
    def consolidate_location(self) -> pd.DataFrame:
        result_df = self.df.copy()
        
        result_df['consolidated_city'] = None
        result_df['consolidated_state'] = None
        result_df['consolidated_country'] = None
        
        state_city_mapping = {
            'illinois': {'state': 'Illinois', 'city': 'Chicago'},
            'il': {'state': 'Illinois', 'city': 'Chicago'},
            'new york': {'state': 'New York', 'city': 'New York City'},
            'n.y.': {'state': 'New York', 'city': 'New York City'},
            'ny': {'state': 'New York', 'city': 'New York City'},
            'california': {'state': 'California', 'city': 'San Francisco'},
            'ca': {'state': 'California', 'city': 'San Francisco'}
        }
        
        city_state_mapping = {
            'chicago': {'state': 'Illinois', 'city': 'Chicago'},
            'new york': {'state': 'New York', 'city': 'New York City'},
            'nyc': {'state': 'New York', 'city': 'New York City'},
            'san francisco': {'state': 'California', 'city': 'San Francisco'}
        }
        
        us_variations = ['united states', 'us', 'usa', 'u.s.']
        
        for idx, row in result_df.iterrows():
            state_val = row.get('State')
            city_val = row.get('City')
            country_val = row.get('Country')
            
            final_state = None
            final_city = None
            final_country = None
            
            # Priority 1: Use State to determine both state and city
            if pd.notna(state_val) and state_val != '':
                state_normalized = str(state_val).strip().lower()
                if state_normalized in state_city_mapping:
                    mapping = state_city_mapping[state_normalized]
                    final_state = mapping['state']
                    final_city = mapping['city']
            
            # Priority 2: If no valid state, use City to determine both city and state
            if final_state is None and pd.notna(city_val) and city_val != '':
                city_normalized = str(city_val).strip().lower()
                if city_normalized in city_state_mapping:
                    mapping = city_state_mapping[city_normalized]
                    final_state = mapping['state']
                    final_city = mapping['city']
            
            # Country logic: 
            # If state or city is filled, use "United States"
            # If neither state nor city is filled, check if country was already filled
            if final_state is not None or final_city is not None:
                final_country = 'United States'
            elif pd.notna(country_val) and country_val != '':
                country_normalized = str(country_val).strip().lower()
                if country_normalized in us_variations:
                    final_country = 'United States'
            
            result_df.at[idx, 'consolidated_state'] = final_state
            result_df.at[idx, 'consolidated_city'] = final_city
            result_df.at[idx, 'consolidated_country'] = final_country
        
        return result_df
    
    def create_clean_dataset(self) -> tuple[pd.DataFrame, pd.DataFrame]:
        consolidated_df = self.consolidate_account_and_email()
        
        self.df = consolidated_df
        
        consolidated_df = self.consolidate_created_date()
        
        self.df = consolidated_df
        
        consolidated_df = self.consolidate_lead_source()

        self.df = consolidated_df
        
        consolidated_df = self.consolidate_monetary('opportunity_amount')

        self.df = consolidated_df
        
        consolidated_df = self.consolidate_is_active()

        self.df = consolidated_df
        
        consolidated_df = self.consolidate_sfdc_id()
        
        self.df = consolidated_df
        
        consolidated_df = self.consolidate_monetary('annual_revenue')

        self.df = consolidated_df
        
        consolidated_df = self.consolidate_last_activity()
        
        self.df = consolidated_df
        
        consolidated_df = self.consolidate_custom_field()
        
        self.df = consolidated_df
        
        consolidated_df = self.consolidate_region()

        self.df = consolidated_df
    
        consolidated_df = self.consolidate_random_notes()

        self.df = consolidated_df

        # Create intermediate cleaned dataset up to notes_flag
        intermediate_df = consolidated_df.copy()
        
        intermediate_df['account_name'] = consolidated_df['consolidated_account_name']
        intermediate_df['contact_email'] = consolidated_df['consolidated_contact_email']
        intermediate_df['created_date'] = consolidated_df['consolidated_created_date']
        intermediate_df['lead_source'] = consolidated_df['consolidated_lead_source']
        intermediate_df['opportunity_amount'] = consolidated_df['consolidated_opportunity_amount']
        intermediate_df['is_active'] = consolidated_df['consolidated_is_active']
        intermediate_df['sfdc_id'] = consolidated_df['consolidated_sfdc_id']
        intermediate_df['annual_revenue'] = consolidated_df['consolidated_annual_revenue']
        intermediate_df['last_activity'] = consolidated_df['consolidated_last_activity']
        intermediate_df['custom_field'] = consolidated_df['consolidated_custom_field']
        intermediate_df['region'] = consolidated_df['consolidated_region']
        intermediate_df['notes_flag'] = consolidated_df['consolidated_random_notes']
        
        # Drop columns for intermediate dataset
        intermediate_columns_to_drop = [
            'consolidated_account_name', 'consolidated_contact_email', 'consolidated_created_date',
            'consolidated_lead_source', 'consolidated_opportunity_amount', 'consolidated_is_active',
            'consolidated_sfdc_id', 'consolidated_last_activity', 'consolidated_annual_revenue',
            'consolidated_custom_field', 'consolidated_region', 'consolidated_random_notes',
            'Account Name', 'AccountName', 'Contact Email', 'Created Date', 'Lead Source', 'Time on Page (sec)',
            'Opportunity Amount', 'Is Active', 'SFDC ID', 'Last Activity', 'Annual Revenue',
            'Custom Field', 'Region', 'Random Notes', 'City', 'State', 'Country', 'Unnamed: 0', 'Unnamed: 21'  
        ]
        
        intermediate_columns_to_drop = [col for col in intermediate_columns_to_drop if col in intermediate_df.columns]
        intermediate_df = intermediate_df.drop(columns=intermediate_columns_to_drop)

        consolidated_df = self.consolidate_deal_score()

        self.df = consolidated_df

        consolidated_df = self.consolidate_engagement_level()

        self.df = consolidated_df

        consolidated_df = self.consolidate_num_calls()

        self.df = consolidated_df

        consolidated_df = self.consolidate_page_duration()

        self.df = consolidated_df
    
        consolidated_df = self.consolidate_location()
        
        clean_df = consolidated_df.copy()
        
        clean_df['account_name'] = consolidated_df['consolidated_account_name']
        clean_df['contact_email'] = consolidated_df['consolidated_contact_email']
        clean_df['created_date'] = consolidated_df['consolidated_created_date']
        clean_df['lead_source'] = consolidated_df['consolidated_lead_source']
        clean_df['opportunity_amount'] = consolidated_df['consolidated_opportunity_amount']
        clean_df['is_active'] = consolidated_df['consolidated_is_active']
        clean_df['sfdc_id'] = consolidated_df['consolidated_sfdc_id']
        clean_df['annual_revenue'] = consolidated_df['consolidated_annual_revenue']
        clean_df['last_activity'] = consolidated_df['consolidated_last_activity']
        clean_df['custom_field'] = consolidated_df['consolidated_custom_field']
        clean_df['region'] = consolidated_df['consolidated_region']
        clean_df['notes_flag'] = consolidated_df['consolidated_random_notes']
        clean_df['deal_score'] = consolidated_df['consolidated_deal_score']
        clean_df['engagement_level'] = consolidated_df['consolidated_engagement_level']
        clean_df['num_calls'] = consolidated_df['consolidated_num_calls']
        clean_df['page_duration'] = consolidated_df['consolidated_page_duration']
        clean_df['city'] = consolidated_df['consolidated_city']
        clean_df['state'] = consolidated_df['consolidated_state']
        clean_df['country'] = consolidated_df['consolidated_country']
            
        columns_to_drop = [
            'consolidated_account_name', 'consolidated_contact_email', 'consolidated_created_date',
            'consolidated_lead_source', 'consolidated_opportunity_amount', 'consolidated_is_active', 'consolidated_engagement_level',
            'consolidated_sfdc_id', 'consolidated_last_activity', 'consolidated_annual_revenue', 'consolidated_deal_score',
            'consolidated_custom_field', 'consolidated_region', 'consolidated_random_notes', 'consolidated_num_calls', 'consolidated_page_duration',
            'consolidated_city', 'consolidated_state', 'consolidated_country',
            'Account Name', 'AccountName', 'Contact Email', 'Created Date', 'Lead Source', 'Time on Page (sec)',
            'Opportunity Amount', 'Is Active', 'SFDC ID', 'Last Activity', 'Annual Revenue', 'Deal Score', 'Engagement Level', 'Num Calls',
            'Custom Field', 'Region', 'Random Notes', 'City', 'State', 'Country', 'Unnamed: 0', 'Unnamed: 21'  
        ]
        
        columns_to_drop = [col for col in columns_to_drop if col in clean_df.columns]
        clean_df = clean_df.drop(columns=columns_to_drop)
        
        return intermediate_df, clean_df
            
            
            

In [110]:
df = pd.read_csv('data/DirtySalesforceData.csv')

print("Original Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

cleaner = Cleansing(df)

# Create both datasets - returns a tuple (intermediate_df, full_clean_df)
intermediate_df, clean_df = cleaner.create_clean_dataset()

# Save intermediate dataset (cleaned up to notes_flag)
intermediate_df.to_csv('IntermediateSalesforceData.csv', index=False)
print(f"\nIntermediate dataset saved as 'IntermediateSalesforceData.csv'")

# Save full cleaned dataset
clean_df.to_csv('CleanSalesforceData.csv', index=False)
print(f"\nFull clean dataset saved as 'CleanSalesforceData.csv'")
print(f"Full clean dataset shape: {clean_df.shape}")

print(f"\nSample of full cleaned data:")
print(clean_df.head(10))



Original Dataset Info:
Shape: (500, 30)
Columns: ['Account Name', 'account_name', 'AccountName', 'Contact Email', 'contact_email', 'Created Date', 'created_date', 'Lead Source', 'lead_source', 'Opportunity Amount', 'opportunity_amount', 'Is Active', 'is_active', 'SFDC ID', 'sfdc_id', 'Annual Revenue', 'annual_revenue', 'Last Activity', 'Custom Field', 'Region', 'Unnamed: 0', 'Unnamed: 21', 'Random Notes', 'Deal Score', 'Engagement Level', 'Num Calls', 'Time on Page (sec)', 'City', 'State', 'Country']

Intermediate dataset saved as 'IntermediateSalesforceData.csv'

Full clean dataset saved as 'CleanSalesforceData.csv'
Full clean dataset shape: (500, 19)

Sample of full cleaned data:
  account_name     contact_email created_date     lead_source  \
0    Acme Corp  contact@acme.com   2020-01-01      Trade Show   
1       Globex   help@globex.com   2022-12-01    Social Media   
2       Globex   help@globex.com         None  Email Campaign   
3    Acme Corp  contact@acme.com         None    

# Written Responses

## Explain your assumptions where data is ambiguous or incorrect:

### Ambiguous Data Assumptions

One thing that caught my eye when I first read the data was the amount of duplicate columns. I knew that eye balling would not be sufficient enough in deciding which columns to keep. I decided to use a python script to help me with this task. This is found in the duplicate-column-insight.ipynb file. Here I creted a ColumnQuality class that had four tests that would give a score for each column on different metrics. The four tests were:

1. Completeness - This would give me a percentage of how much column was filled with data. The higher the better.
2. Quality - This would give me a percentage of how much of a columns had valid data. I would consider invalid data to be anything with one of these values - 'not_a_date', 'noemail', 'invalid@', 'BAD_ID', 'N/A', 'NaT', 'ERROR'. The higher the better.
3. Format Consistency - This would give me a percentage of how much of a columns had consistent data. This could only apply to columns with a date, boolean or email. The higher the better.
4. Data Type - This would give me a percentage of how much of a columns had consistent data. This could only apply to columns with values that were numeric or strings. The higher the better.

I then compared these scores for each columns and decided to prioritize using data from the columns with the best score. 

I also found the Random Notes column to be quite ambigous. So I ran a pythons cript in the same file to see what percentage each of the values occurred. Analyzing if one value stood out allowed me to discern the significance of the column and its meaning. Unfortunately I could not discern any meaning from the column, but figure it would be bad practice to completely remove it. I will get into what I did with this column later. 

I also found the deal score column to be amigous. I was worried that this column could be correlated to the Opportunity Amount columns and the Annual Revenue columns. So I created a script that would find the correlation between these columns and the deal score column. I found that the correlation was not significant enough to justify using a different strategy when cleaning this column later on. 

### Incorrect Data Assumptions

Making assumpitons for incorrect data was much easier as I could clearly see which data was just wrong and not valid record value. For instance Tthere were various malformed email addresses that had values with no domain, a missing @ symbol, or jst containing the value no email. Recognizing these things allowed me to determine which email addresesses were wellformed and which were not. 

I also saw invalid date formats that were clearly inconsistent or contained values like NaT and not_a_date. I made the conclusion that the best formed date values would be in the format YYYY-MM-DD as this is commonly used in salesforce systems, therfore anything else would have to be converted.

Overall accross these columns I saw malformed data of all sorts. Numeric columns often mixed strings and numbers. Boolean columns used strings of binary values. Locational columns often mixed Abbreviations, full names, and even the casing varied. Across all of these place holder values that were not valid records even varied within the same column. To resolve these issues I defualted to the way data like this would be represented in salesforce systems. 

## Identify and describe gaps or potential data quality risks:
    
Like I menitoned the duplicate columns flagged a potential risk. This suggested to me that there were integration issues where multiple stytems were not properly consolidated. Like I mentioned I used a script to help me with this task. Something else that was alarming was the amount of missing data within the columns that told me the data was not being collected consistently or properly from the source. 

There wer also strucutral gaps like an unamed column and one that just contained row indexes. This suggested that the software or scripting used for collection was not consistent or formattically sound. The columns themselves were also inconsistent with naming. The mixed CamelCase, snake_case, and just spaces. Within the columns there was also mixing of data types, where at times the same column would contain strings and numbers. 

Additionally like I mentioned before I found that the valus of the data itself was was inonsistent. For instance the Boolean columns contaiend various ways of reprpesenting true and false values. Additioanlly string values mixed different reprsenations of the values where at times full names were used and at other times abbreviations. I also found that data simply did not make sense. An example would be a record that had a state value of Illinois but a city value of New York City. These gaps highlighted the disregard for data quality and consistency. 


## Justify each of transformation or cleaning step in your code:

1. Duplicate Column Consolidation Strategy

Implementation: Created a duplicate_groups dictionary to map final fields to their various column representations.

Justification: Rather than arbitrarily choosing columns, I used my ColumnQualityAnalyzer to score each duplicate column on completeness, quality, format consistency, and data type consistency. This approach allowed me to identify the most reliable columns to use for each final field, and create a prioritization order. Anytime a column was not found in the data I would use the next best column based on the priority order that was determined by the ColumnQualityAnalyzer. 

2. Account Name and Contact Email Consolidation

Implementation: Consolidated account names and emails together, with logic to derive missing information from valid counterparts.

Justification: These fields are almost always consistent with each other in CRM Systems.. When an account name was missing but a valid email existed (help@globex.com or contact@acme.com), I could reliably infer the account name. Conversely, when account names existed without valid emails, I generated the corresponding corporate emails. I would always priortiize to infer based on the account name as its scores were higher than the email fields. I would only infer in the other direction if all of the account names were missing. Laslty many of the account names did not have a valid email so we simply used None. 

3. Date Standardization 

Implementation: Standardized all dates to YYYY-MM-DD format using pandas to_datetime.

I chose ISO 8601 format (YYYY-MM-DD) as it's universally recognized, sorts correctly, and is commonly used in Salesforce systems. Again anything that was not a valid date was set to None.

4. Lead Source Consolidation

Implementation: Consolidated lead sources to a single field using the Duplicate Column Strategy.

Justification: This was a catergory where any valid value is prefered to null. Still I prioritized lead_source since it was the highest scoring field. 

5. Is Active Consolidation

Implementation: Mapped various string representations to proper boolean values.

Justification: Here I simply mapped the various represenations of boolean values to a standardized boolean value of True or False. 

6. SFDC ID Consolidation

Implementation: Filtered out obvious placeholder values and kept legitimate IDs. I did this again following the Duplicate Column Strategy.

Justification: Placeholder patterns like ('abc123', 'xyz-00001', '12345', 'bad_id') clearly were not valid SFDC IDs and were filtered out. This would prevent false linking in systems. I decided between the columns which value to prioritize based on the scores from the Duplicate Column Strategy.

7. Monetary Value Standardization

Implementation: I parsed any monetary values by:
    - Decided between the columns which value to prioritize based on the scores from the Duplicate Column Strategy
    - Removing the currencey symbol ($)
    - Converting text to number (five million -> 5000000.0) through Dictionary-based parsing of written numbers
    - Handling Suffixes like K, M, and B

Justification: Converting these values to a standard number format would allow easier comparison and analysis with this data down the line.

9. Last Activity Consolidation

Implementation: Here I filtered out placeholder values and kept legitimate dates. I again used the Duplicate Column Strategy to determine which column to prioritize.

Justification: I decided to use date values and filter anuthing else out as other possible values such as 'Called Client' were not varied enough to provide meaning to this column. After filtering I standardized the dates.

10. Preserving Custom Field

Implementation: Here I kept any JSON values that were properly formatted. I got rid of any nulls or JSON values that represented nulls.

Justification: Custom fields can contain data that is later on parsed seperatley and therefore I wanted to preserve values that were valid. I opted not to parse the valid JSON values as there was not enough context to try and give this data new meaning. 

11. Region Standardization

    Implementation: Mapped various North America representations to a single standard value.

    Justification: To keep consistencey standardized all variations ('North America', 'NA', 'N.A.', 'United States', 'US') to 'North America' as the most descriptive option.

12. Location Data Standardization

Implementation: I used the ColumnQualityAnalyzer to score State and City and to determine which column was basis for the ones that were related. State scored better so if a State and City were conflicting geographically I would populate the city based on the state. If either column was missing I would use the other column to infer the missing value. I also ensured format consistencey by making sure that cities, states and countries were all fully spelled out. 

Justification: I needed to enusre the the locational data was not only consistent between their respective columns, but also made sense across the records.

13. Random Notes Flag

Implementation: Converted the values 'See notes' and 'Valid' to a standard 'Notes_Flag'

Justification: Since the values within the random notes column were not consistent and ambigous I tried to maintain information by marking these records in a standardized way.

14. Deal Score Normaliation

Implementation: Since the values ranged between 0 and 100 I decided to standardize the values to be between 0 and 1. 

Justification: I used scores elswhere in my code, and decimals were also used for the engagement level, so I chose to do the same here. 

15. Engagement Level Normalization

Implementation: I rounded the values in this column to 4 decimal places.

Justification: I wanted to preserve the accuracy of these values, but thought there need to be a standard to which the exactness of the data was measured. 

16. General Blank Values and Useless Columns

Implementation: I dropped all columns that were not needed for the analysis, and filled all blank values across columns with None. Certain columns only needed there blank values to be filled with None and no further cleaning was needed.

Justification: Columns deemed useless were dropped as they provided no apparent valuable information. As far as populating blanks with None, I wanted to make sure that even records without a value for a given column could be recognized as such.

17. Column Naming

Implementation: All output columns used snake_case naming.

Justification: This would help with readability and also usage of the dataset down the line with things such as API integrations. 


# Moving Forward

## Root Cause and Prevention

I would start by tracking bad data back to the source. We should look at which system created this data and what changes were made to it along the way. I would also see how data changed over time and what patterns emerged. Next I would compare the error rates of data from different sources. For example we could analyze if manual entry was more prone to errors than automated entry. Finally I would look for patters in the errors to see if users, time periods, or processes kept producing bad data. This could tell us if it is a system problem or just not great training. 

## People Systems and Processes to Review

The Salesforce team should check how the they set up field mappins and data rules. We could then see if they are accumulating data properly. We also need to review if data or IT teams are properly transferring data. We should check on their system connections and any programs that alter data between systems. The Sales team could be intervies to see how they actually enter data. We could also take a look at their training and test them to see if they are following proper procedures. 

## Data Producers Recommendations

We should setup basic data checks that stop bad data from being submitted. THis could be making sure that emails include an @ sign, IDs follow the correct format, etc. While I am not to knowledgable of the logistics of this, I would suggest setting up some sort of automatic monriotring to show data quality problems. Lastly, setting up better training that is simplified and easily understandable, especially for those that submitting data. Settuping clear rules and defining processes would make it easier for users to follow proper procedures.










    






